In [1]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

## Fine Food Reviews

Fuente: https://inclass.kaggle.com/c/fine-food-reviews

### Representacion

- **Id** - El id que identifica a cada review
- **ProductId** - El Id del producto
- **UserId** - El Id del usuario
- **ProfileName** - El nombre del usuario 
- **HelpfulnessNumerator** - El numerador indicando la cantidad de usuarios que juzgaron al review como util
- **HelpfulnessDenominator** - El denominador indicando la cantidad de usuarios que evaluaron si el review fue útil o no
- **Prediction** - La cantidad de estrellas del review
- **Time** - Un timestamp para el review
- **Summary** - Un resumen del review
- **Text** - Texto del review

In [37]:
## creamos el RDD a partir de
reviewsTextRDD = sc.textFile('data/fine-food-reviews-train.csv',8)
## aplicamos una accion para tomar los 15 items del RDD
reviewsTextRDD.take(1)

['"Id","ProductId","UserId","ProfileName","HelpfulnessNumerator","HelpfulnessDenominator","Prediction","Time","Summary","Text"']

### Parseando el CSV

In [51]:
# esto puede mejorarse mucho y probablemente acarre errores de procesamiento
reviewsRDD = reviewsTextRDD.map(lambda a: a.split(','))
reviewsRDD.take(1)

[['"Id"',
  '"ProductId"',
  '"UserId"',
  '"ProfileName"',
  '"HelpfulnessNumerator"',
  '"HelpfulnessDenominator"',
  '"Prediction"',
  '"Time"',
  '"Summary"',
  '"Text"']]

### User Profiles con Mayor Cantidad de Reviews

In [52]:
from operator import add
reviewsRDD.map(lambda a: (a[3], 1)).reduceByKey(add).sortBy(ascending=False,keyfunc=lambda x:x[1]).take(10)

[('"C. F. Hill ""CFH"""', 347),
 ('"O. Brown ""Ms. O. Khannah-Brown"""', 338),
 ('"Gary Peterson"', 306),
 ('"Chris"', 291),
 ('"Rebecca of Amazon ""The Rebecca Review"""', 289),
 ('"Linda"', 224),
 ('"Mike"', 209),
 ('"c2"', 207),
 ('"Laura"', 206),
 ('"John"', 204)]

### ProductIds con Mayor Cantidad de Reviews

In [53]:
from operator import add
reviewsRDD.map(lambda a: (a[1], 1)).reduceByKey(add).sortBy(ascending=False,keyfunc=lambda x:x[1]).take(5)

[('"B007JFMH8M"', 714),
 ('"B003B3OOPA"', 511),
 ('"B0026RQTGE"', 509),
 ('"B002QWP8H0"', 508),
 ('"B002QWP89S"', 499)]

### Cantidad totales de Reviews por Calificacion

In [54]:
reviewsPerScoreRDD = reviewsRDD.map(lambda a: (a[6], 1)).reduceByKey(add).sortBy(ascending=False,keyfunc=lambda x:x[1])
reviewsPerScoreRDD.take(5)

[('5', 288513), ('4', 64077), ('1', 42194), ('3', 33936), ('2', 24035)]

### Probabilidad total de cada Calificacion

In [56]:
totalFreq = reviewsRDD.count()
reviewsPerScoreRDD.map(lambda x: (x[0], round(float(x[1])/totalFreq,3))).take(6)

[('5', 0.634),
 ('4', 0.141),
 ('1', 0.093),
 ('3', 0.075),
 ('2', 0.053),
 ('0', 0.003)]